### Create dashboard

In [1]:
from get_data import get_data
import numpy as np
import pandas as pd 

import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html

import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots

In [2]:
df = get_data()

In [3]:
external_css = ["https://fonts.googleapis.com/css?family=Overpass:300,300i",
                "https://cdn.rawgit.com/plotly/dash-app-stylesheets/dab6f937fd5548cebf4c6dc7e93a10ac438f5efb/dash-technical-charting.css"]
    
    

app = dash.Dash(__name__, external_stylesheets = external_css)

In [4]:
app.layout = html.Div(
[

        html.Div([
            html.H1(
                'Cost of Type II Errors in Recession Forecasting',
                className='twelve columns',
                style={'text-align': 'center'}
                    ),
                ],
            className='row'
                ),

    
        html.Div([
            
            html.Div([
                html.Label('Stock fund:'),
                dcc.Dropdown(
                    id='stock_dropdown',
                    options=[{'label': 'SPX', 'value': 'Adj Close'}],
                    value='Adj Close',
                    placeholder='SPX',
                    ),
                    ],
                    className = 'four columns',
                    style = {'marginLeft' : 20},
                    ),
        
            html.Div([
                html.Label('Stock %:'),
                dcc.Input(
                    id='stock_perc',
                    type = 'number',
                    placeholder = 1,
                    value = 1,
                    min = 0,
                    max = 100, 
                    ),
                    ],
                    className='three columns',
                    ),

            html.Div([
                html.Label('Recession Forecast:'),
                dcc.Dropdown(
                    id='recess_forecast',
                    options=[{'label': 'OECD US P->T', 'value': 'USARECM'},
                             {'label': 'OEC US P<-T', 'value': 'USARECP'},
                             {'label': 'Smoothed Recess Probabilities', 'value': 'RECPROUSM156N'},
                             {'label': 'SPF Current Qtr', 'value': 'recess1'},
                             {'label': 'SPF +1 Qtr', 'value': 'recess2'},
                             {'label': 'SPF +2 Qtr', 'value': 'recess3'},
                             {'label': 'SPF +3 Qtr', 'value': 'recess4'},
                             {'label': 'SPF +4 Qtr', 'value': 'recess5'}],
                    value='recess1',
                    placeholder='SPF Current Qtr',
                    ),
                    ],
                    className='four columns',
                    style = {'marginRight' : 20},
                    ),
        
                ],
                className='row',
                ),
   
            html.Div([            
                html.Br(),
            ], className='row'
            ),    
    
        html.Div([
            
            html.Div([
                html.Label('Bond fund:'),
                dcc.Dropdown(
                    id='bond_dropdown',
                    options=[{'label': 'BBG AGG', 'value': 'LBUSTRUU'}],
                    value='LBUSTRUU',
                    placeholder='BBG AGG',
                    ),
                    ],
                    className = 'four columns',
                    style = {'marginLeft' : 20},
                    ),
        
            html.Div([
                html.Label('Bond %:'),
                dcc.Input(
                    id='bond_perc',
                    type = 'number',
                    placeholder = 0,
                    value = 0,
                    min = 0,
                    max = 100, 
                    ),
                    ],
                    className='three columns',
                    ),
            
            html.Div([
                html.Label('Recession Indicator Threshold:'),
                dcc.Input(
                    id='recess_thresh',
                    type = 'number',
                    placeholder = 10,
                    value = 10,
                    min = 0,
                    max = 100, 
                    ),
                    ],
                    className='four columns',
                    style = {'marginRight' : 20},
                    ),            
        
        
                    ],
                    className='row'
                    ),
    
            html.Div([            
                html.Br(),
            ], className='row'
            ),    
                    
        html.Div([        
            html.Div([
                html.Label('Starting $'),
                dcc.Input(
                    id='starting_money',
                    type = 'number',
                    placeholder = 10000,
                    value = 10000,
                    min = 0,
                    ),
                    ],
                    className='four columns',
                    style = {'marginLeft' : 20},
                    ),              
       
            html.Div([
                html.Label('$ Contribution'),
                dcc.Input(
                    id='contribution',
                    type = 'number',
                    placeholder = 100,
                    value = 100,
                    min = 0,
                    ),
                    ],
                    className='four columns',
                    ),
                ],
                 className='row'
        ),
            
            html.Div([            
                html.Br(),
                html.Br(),
            ], className='row'
            ),
    
        html.Div([
            dcc.Graph(id='returns_plot', style={'max-height': '600', 'height': '60vh'}),
        ],
            className='row',
            style={'marginBottom': '20', 'marginTop' : '20'}
        ),
    
        html.Div([
            dcc.Graph(id='growth_plot', style={'max-height': '600', 'height': '60vh'}),
        ],
            className='row',
            style={'marginBottom': '20', 'marginTop' : '20'}
        ),
    
    
            html.P(
            hidden='',
            id='returns_json',
            style={'display': 'none'}
        ),
            
            
            
            
            
            
],
    
    
    
    
    
     style={
        'width': '85%',
        'max-width': '1200',
        'margin-left': 'auto',
        'margin-right': 'auto',
        'font-family': 'overpass',
        'background-color': '#F3F3F3',
        'padding': '40',
        'padding-top': '20',
        'padding-bottom': '20',
    },
)


In [5]:
#Produce returns DF

@app.callback(Output('returns_json', 'hidden'),
              [Input('stock_dropdown', 'value'),
               Input('stock_perc', 'value'),
               Input('bond_dropdown', 'value'),
               Input('bond_perc', 'value'),
               Input('recess_forecast', 'value'),
               Input('recess_thresh', 'value')])

def calc_returns(stock, stock_perc, bond, bond_perc, forecast, threshold):
    
  
    df['stock returns'] = (df[stock] - df[stock].shift()) / df[stock].shift()
    df['bond returns'] = (df[bond] - df[bond].shift()) / df[bond].shift()
    
    df['allocated_non_recess_return'] = (df['stock returns'] * stock_perc) + (df['bond returns'] * bond_perc)
    df['recession_ind'] = np.where(df[forecast] >= threshold, 1, 0)
    df['recession_return'] = np.where(df['recession_ind'] == 1, (df['stock returns'] * bond_perc) + (df['bond returns'] * stock_perc), (df['stock returns'] * stock_perc) + (df['bond returns'] * bond_perc))

    df2 = df[['date', 'stock returns', 'bond returns', 'allocated_non_recess_return', 'recession_ind', 'recession_return']]
    
    df2.dropna(inplace=True)
    df2.reset_index(inplace=True)
    
    return df2.to_json()
    
    

In [6]:
#Produce plot of returns

@app.callback(Output('returns_plot', 'figure'),
              [Input('returns_json', 'hidden')])
              
def ret_plot(json):
    
    df = pd.read_json(json)

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=df['date'], y=df['stock returns'], name = 'stock returns'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df['date'], y=df['bond returns'], name = 'bond returns'), secondary_y=False)
    #fig.add_trace(go.Scatter(x=df['date'], y=df['gold returns'], name = 'gold returns'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df['date'], y=df['allocated_non_recess_return'], name = 'allocated returns'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df['date'], y=df['recession_return'], name = 'recess allocated returns'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df['date'], y=df['recession_ind'], name = 'recession indicator'), secondary_y = True)

    return fig
        
              
              


In [7]:
# Produce growth of 10k

@app.callback(Output('growth_plot', 'figure'),
              [Input('returns_json', 'hidden'),
               Input('starting_money', 'value'),
               Input('contribution', 'value')])

def growth_plot(json, start, contribution):
    
    df = pd.read_json(json)
    
    length = list(range(len(df.index)))
    
    ret_cols = ['stock returns', 'bond returns', 'allocated_non_recess_return', 'recession_return']
    
    growth_cols = ['stock growth', 'bond growth',  'allocated_growth', 'allocated_recess_growth']

    for ret, growth in zip(ret_cols, growth_cols):
        for x in length:
            if x == 0:
                df.loc[x, growth] = start * (1 + df.loc[x, ret]) 
                print(start * (1 + df.loc[x, ret]) )
            else:
                df.loc[x, growth] = (df.loc[x-1, growth] + contribution) *  (1 + df.loc[x, ret])
                print((df.loc[x-1, growth] + contribution) *  (1 + df.loc[x, ret]))
    print(start)
    print(contribution)
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=df['date'], y=df['stock growth'], name = 'stock growth'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df['date'], y=df['bond growth'], name = 'bond growth'), secondary_y=False)
    #fig.add_trace(go.Scatter(x=df['date'], y=df['gold growth'], name = 'gold growth'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df['date'], y=df['allocated_growth'], name = 'allocated growth'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df['date'], y=df['allocated_recess_growth'], name = 'recess allocated growth'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df['date'], y=df['recession_ind'], name = 'recession indicator'), secondary_y = True)
    
    return fig

In [8]:
# Produce efficient frontier



In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jan/2020 01:46:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2020 01:46:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2020 01:46:51] "GET /_dash-dependencies HTTP/1.1" 200 -
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

127.0.0.1 - - [30/Jan/2020 01:46:51] "POST /_dash-update-component HTTP/1.1" 200 -


9754.601252
10068.949896933667
11554.192383276966
10415.738225578309
11446.945338313042
11710.49971076523
10095.002922440211
11293.003174293075
11901.622574705176
13198.356862726981
14046.782582228485
16172.749632367848
15886.146186699201
15585.776586080292
14214.103456075189
15181.624738721946
16607.173996933052
16633.24499759208
15692.059559681886
18860.66436454796
20465.489038884905
24270.290808740257
29950.756584555136
32427.191375664475
39678.56018253776
43740.45705040201
39650.39060672795
36405.462141054224
42745.70264886864
44643.971596890486
47251.518430000935
39548.834695831756
40519.362551761005
39984.704976306435
36979.85402591595
39006.060191593446
41640.71147007984
46710.35346926439
47785.566262738466
41777.71726870533
44326.69521857061
49703.913879304455
64437.69324647664
59763.175913179795
52908.087894690056
54501.28802072975
51895.98621565217
52941.38140710325
57966.647580666875
58155.415911342825
59021.94440225089
55282.87363006
59613.44986986527
68885.13132682376
6214

127.0.0.1 - - [30/Jan/2020 01:46:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2020 01:46:53] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\pandas\core\ops\__init__.py", line 968, in na_op
    result = expressions.evaluate(op, str_rep, x, y, **eval_kwargs)
  File "C:\Anaconda\lib\site-packages\pandas\core\computation\expressions.py", line 221, in evaluate
    return _evaluate(op, op_str, a, b, **eval_kwargs)
  File "C:\Anaconda\lib\site-packages\pandas\core\computation\expressions.py", line 127, in _evaluate_numexpr
    result = _evaluate_standard(op, op_str, a, b)
  File "C:\Anaconda\lib\site-packages\pandas\core\computation\expressions.py", line 70, in _evaluate_standard
    return op(a, b)
TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Anaconda\lib\sit

...

127.0.0.1 - - [30/Jan/2020 01:49:16] "POST /_dash-update-component HTTP/1.1" 200 -



1891083.0566233126
1911131.9815297509
1879549.7884619837
1949256.064790721
2119512.8244474526
2228358.5654594693
2366970.796280339
2419233.548165187
2467276.424566216
2451775.4400550826
2697981.331554788
2927418.327298234
2996102.3568642214
3029463.637572176
2853806.476958093
2932667.1227135966
2872997.203523537
2939772.67500375
3067678.557391644
2931741.1384315966
2813344.122369763
2945346.7286271024
2950551.468214342
2914990.5336806113
3099388.6789100203
3024433.6269842237
3308972.855302193
3410780.3170602242
3401781.2743373937
3284247.256390032
3249384.393227844
3209406.588259857
3309655.8928682823
3090651.1029459466
3100766.8151064157
3405047.7152972384
3373218.763761921
3355635.8375969008
2941515.5179931596
2931105.8052605544
3100882.232886591
3247967.448310207
3110551.706075915
3193421.2427879195
3268466.667421239
3210094.0320032705
3222973.800319822
3269968.757989833
3272088.48620058
3356796.813704416
3483716.909330209
3783226.917279763
3729771.966137323
3863132.0805472634
3937

127.0.0.1 - - [30/Jan/2020 01:49:16] "POST /_dash-update-component HTTP/1.1" 200 -


9754.601252
9966.774784623667
11324.476417870866
10121.060132792429
11017.317055691814
11173.369615425652
9550.417144366396
10578.99560673571
11051.275155871712
12153.24614546685
12837.225533641782
14675.650334135822
14326.990328003867
13968.173929811135
12657.64995703041
13424.780134600576
14589.264124576846
14524.706845287776
13620.942317881147
16267.670497032954
17558.764073041304
20721.91473244343
25466.951912397057
27480.896232828374
33522.7958187669
36861.62606166566
33338.56373951247
30533.179712969126
35752.51878450416
37253.08104323606
39340.82252124257
32858.15826562601
33579.59036531307
33054.92577022372
30494.582173995943
32078.69985046404
34157.874208346264
38224.704894505994
39021.05054193567
34043.87886162216
36034.73973221456
40315.121156355264
52160.82849559456
48301.9552194733
42690.085403147306
43892.63473922721
41717.90904512709
42476.42741316189
46420.663131801375
46491.62787570804
47103.367690760795
44044.720618143954
47409.20021847377
54691.01166096495
49268.3692

127.0.0.1 - - [30/Jan/2020 01:49:51] "POST /_dash-update-component HTTP/1.1" 200 -
